# Plotting a stock's price chart with corporate action overlay

Sometimes, a picture is worth a thousand words.

In [1]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
equitiesIndiaNse <- EquitiesIndiaNse()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date


Attaching package: ‘PerformanceAnalytics’

The following obje

In [2]:
symbol <- "YESBANK"
startDt <- as.Date(Sys.Date() - 3*365) #3-year chart
endDt <- Sys.Date()

startDtNum <- as.numeric(format(startDt, '%Y%m%d'))
endDtNum <- as.numeric(format(endDt, '%Y%m%d'))

# examples: https://github.com/shyams80/plutons/blob/master/docs-R/EquitiesIndiaNse.ipynb

#make sure that the data exists

equitiesIndiaNse$EodTimeSeries() %>%
    filter(TIME_STAMP >= startDt & TIME_STAMP <= endDt 
           & SYMBOL == symbol 
           & (SERIES == 'EQ' | SERIES == 'BE')) %>%
    summarize(MAX_DT = max(TIME_STAMP), MIN_DT = min(TIME_STAMP)) %>%
    print()

equitiesIndiaNse$EodAdjustedTimeSeries() %>%
    filter(DATE_STAMP >= startDtNum & DATE_STAMP <= endDtNum 
           & SYMBOL == symbol) %>%
    summarize(MAX_DT = max(DATE_STAMP), MIN_DT = min(DATE_STAMP)) %>%
    print()


Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:   lazy query [?? x 2]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
  MAX_DT     MIN_DT    
  <date>     <date>    
1 2019-07-18 2016-07-19
# Source:   lazy query [?? x 2]
# Database: postgres [ro1@SWEDEN:5432/StockVizDyn]
    MAX_DT   MIN_DT
     <int>    <int>
1 20190718 20160719


In [3]:
# so the data exists in both so we can use the EodAdjustedTimeSeries

eodPxDt <- equitiesIndiaNse$EodAdjustedTimeSeries() %>%
    filter(DATE_STAMP >= startDtNum & DATE_STAMP <= endDtNum & SYMBOL == symbol) %>%
    collect() %>%
    mutate(TIME_STAMP = as.Date(as.character(DATE_STAMP), format="%Y%m%d")) %>%
    select(TIME_STAMP, CLOSE) %>%
    arrange(TIME_STAMP)

print(eodPxDt)

# A tibble: 739 x 2
   TIME_STAMP CLOSE
   <date>     <dbl>
 1 2016-07-19  224.
 2 2016-07-20  226.
 3 2016-07-21  224.
 4 2016-07-22  225.
 5 2016-07-25  230.
 6 2016-07-26  233.
 7 2016-07-27  233.
 8 2016-07-28  238.
 9 2016-07-29  236.
10 2016-08-01  237.
# … with 729 more rows


In [4]:
corpEodDt <- equitiesIndiaNse$CorporateActions() %>%
    filter(EX_DATE >= startDt & EX_DATE <= endDt & SYMBOL == symbol & SERIES == 'EQ') %>%
    select(TIME_STAMP = EX_DATE, PURPOSE) %>%
    collect() %>%
    full_join(eodPxDt, by=c('TIME_STAMP')) %>%
    arrange(TIME_STAMP)

print(corpEodDt)

# A tibble: 739 x 3
   TIME_STAMP PURPOSE CLOSE
   <date>     <chr>   <dbl>
 1 2016-07-19 NA       224.
 2 2016-07-20 NA       226.
 3 2016-07-21 NA       224.
 4 2016-07-22 NA       225.
 5 2016-07-25 NA       230.
 6 2016-07-26 NA       233.
 7 2016-07-27 NA       233.
 8 2016-07-28 NA       238.
 9 2016-07-29 NA       236.
10 2016-08-01 NA       237.
# … with 729 more rows


In [5]:
plotData <- data.frame(corpEodDt)
plotData$EX_DATE <- ifelse(is.na(plotData$PURPOSE), NA, plotData$TIME_STAMP)

startDate <- first(plotData$TIME_STAMP)
endDate <- last(plotData$TIME_STAMP)

xAxisTicks <- seq(from=startDate, to=endDate, length.out=10)

ggplot(plotData, aes(x = TIME_STAMP, y = CLOSE)) + 
    theme_economist() +
    geom_line() + 
    geom_vline(xintercept=EX_DATE, colour="grey") +
    scale_y_log10() +
    scale_x_date(breaks = xAxisTicks) +
    labs(x='', y='log(closing price)', fill='', color='',
         title=sprintf("%s Closing Price", symbol), 
         subtitle=sprintf("[%s:%s]", startDate, endDate)) +
    annotate("text", x=startDate, y=min(plotData$CLOSE, na.rm=T), label = "@StockViz", 
             hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

ERROR: Error in new_data_frame(list(xintercept = xintercept)): object 'EX_DATE' not found


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)